In [88]:
import pandas as pd
import numpy as np

Tref = 18
df = pd.read_csv('/home/pguser/code/raw_data/europeweather.csv')
df['date'] = pd.to_datetime(df['date'])

df.set_index("date", inplace=True)

temperature_df = df.groupby(['country', pd.Grouper(freq='M', level='date')])['avg_temp'].mean().reset_index()
temperature_df['month'] = pd.to_datetime(temperature_df['date'].dt.strftime('%Y-%m'), format='%Y-%m')
temperature_df = temperature_df[~(temperature_df['month'] < '2015-01-01')]
temperature_df['avg_temp'] = (5/9) * (temperature_df['avg_temp'] - 32)
temperature_df['HDD'] = np.maximum(0, Tref - temperature_df['avg_temp'])


In [92]:
import sqlalchemy
import pandas as pd
pd.set_option('display.max_rows', None)

url_object = sqlalchemy.engine.URL.create(
    "postgresql+psycopg2",
    username="pguser",
    password="pgpass",
    host="localhost",
    database="finance_dwh",
    port=5431
)

engine = sqlalchemy.create_engine(
    url_object)

query_str = """select 
country_code country, 
amount, 
time_period from 
eu_lng_consumptions 
where 
energy_balance = 'IC_OBS' and 
unit = 'MIO_M3' 
order by time_period desc
"""

df = pd.read_sql_query(query_str, con=engine)
df = df[df['amount'].notna()]
df['month'] = pd.to_datetime(df['time_period'], format='%Y-%m')
df = df[~(df['month'] < '2015-01-01')]
gas_consumption_df = df


In [108]:
from sklearn.linear_model import LinearRegression

combined_df = temperature_df.merge(gas_consumption_df, on=['country', 'month'], how='inner')
# Create a new column 'A_Coefficient' in both dataframes and initialize with NaN
temperature_df['A_Coefficient'] = np.nan
gas_consumption_df['A_Coefficient'] = np.nan

# Group the data by 'Country' and calculate the coefficient for each group
for country, group in combined_df.groupby('country'):
    regression_model = LinearRegression()
    X = group[['HDD']].values
    y = group['amount'].values
    regression_model.fit(X, y)
    a_coefficient = regression_model.coef_[0]

    # Assign the coefficient to the respective rows in both dataframes
    temperature_df.loc[temperature_df['country'] == country, 'A_Coefficient'] = a_coefficient
    gas_consumption_df.loc[gas_consumption_df['country'] == country, 'A_Coefficient'] = a_coefficient
    combined_df.loc[combined_df['country'] == country, 'A_Coefficient'] = a_coefficient

combined_df[combined_df['country'] == 'FR'].sort_values(by=['month'], ascending=False)


,country,date,avg_temp,month,HDD,A_Coefficient_x,amount,time_period,A_Coefficient_y,A_Coefficient
629,FR,2023-09-30 00:00:00+00:00,20.161943,2023-09-01,0.000000,323.688699,1401.760,2023-09,323.688699,323.688699
628,FR,2023-08-31 00:00:00+00:00,21.033189,2023-08-01,0.000000,323.688699,1280.547,2023-08,323.688699,323.688699
627,FR,2023-07-31 00:00:00+00:00,21.195413,2023-07-01,0.000000,323.688699,1387.517,2023-07,323.688699,323.688699
626,FR,2023-06-30 00:00:00+00:00,20.618798,2023-06-01,0.000000,323.688699,1433.547,2023-06,323.688699,323.688699
625,FR,2023-05-31 00:00:00+00:00,15.403928,2023-05-01,2.596072,323.688699,1693.354,2023-05,323.688699,323.688699
624,FR,2023-04-30 00:00:00+00:00,11.292954,2023-04-01,6.707046,323.688699,2704.045,2023-04,323.688699,323.688699
623,FR,2023-03-31 00:00:00+00:00,9.774339,2023-03-01,8.225661,323.688699,3786.393,2023-03,323.688699,323.688699
622,FR,2023-02-28 00:00:00+00:00,6.341096,2023-02-01,11.658904,323.688699,4369.696,2023-02,323.688699,323.688699
621,FR,2023-01-31 00:00:00+00:00,6.278113,2023-01-01,11.721887,323.688699,4836.159,2023-01,323.688699,323.688699
620,FR,2022-12-31 00:00:00+00:00,6.673447,2022-12-01,11.326553,323.688699,4944.227,2022-12,323.688699,323.688699


In [111]:
def calculate_expected_gas_consumption(current_temperature, average_temperature, average_gas_consumption):
    # Calculate the Heating Degree Days (HDD)
    hdd = max(0, average_temperature - current_temperature)
    
    # Calculate the expected gas consumption
    expected_gas_consumption = average_gas_consumption + (hdd * 323.77323730744469)
    
    return expected_gas_consumption

# Input data
current_temperature = 8  # Current temperature in Celsius
average_temperature = 10  # Average temperature for this time of year in Celsius
average_gas_consumption = 3538.257  # Average gas consumption for this time of year

# You need to define the coefficient 'a_coefficient' based on your specific application
# The coefficient represents the relationship between HDD and gas consumption

# Calculate the expected gas consumption
expected_gas = calculate_expected_gas_consumption(current_temperature, average_temperature, average_gas_consumption)

print(f"Expected Gas Consumption: {expected_gas} units")

Expected Gas Consumption: 3732.5209423844667 units
